<a href="https://colab.research.google.com/github/ronakdm/input-marginalization/blob/main/lstm_snli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ronakdm/input-marginalization.git

fatal: destination path 'input-marginalization' already exists and is not an empty directory.


In [ ]:
%%bash
cd input-marginalization
git pull
cd ..

Updating 7baf1ee..609e722
Fast-forward
 models.py | 16 ++++++++++++----
 1 file changed, 12 insertions(+), 4 deletions(-)


From https://github.com/ronakdm/input-marginalization
   7baf1ee..609e722  main       -> origin/main


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
save_dir = "/content/gdrive/My Drive/input-marginalization"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pickle
import numpy as np
import time
import datetime
import random
import torch
import torch.nn as nn

In [ ]:
import sys
sys.path.append("input-marginalization")

from utils import generate_dataloaders, train, test
from models import LSTM

In [ ]:
try:
    from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup, BertTokenizer
except ModuleNotFoundError:
    !pip install transformers
    from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup, BertTokenizer
    

In [ ]:
LEARNING_RATE = 1e-4
ADAMW_TOLERANCE = 1e-8
BATCH_SIZE = 10
EPOCHS = 30
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
VOCAB_SIZE = tokenizer.vocab_size
LABEL_SIZE =2
HIDDEN_DIM = 200
EMBEDDING_DIM = 100
N_RNN_LAYERS = 2

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print("Running on '%s'." % device)

Running on 'cuda'.


In [ ]:
train_dataloader, validation_dataloader, test_dataloader = generate_snli_dataloader(pre, BATCH_SIZE)

6,919 training samples.
  876 validation samples.
1,822 test samples.


In [ ]:
model = SNLILSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, LABEL_SIZE, N_RNN_LAYERS).to(device)
save_filename = "lstm_snli2"
optimizer = AdamW(model.parameters(), lr = LEARNING_RATE, eps = ADAMW_TOLERANCE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = EPOCHS * BATCH_SIZE * len(train_dataloader))

In [ ]:
try:
    train(model, EPOCHS, train_dataloader, validation_dataloader, optimizer, scheduler, save_dir, save_filename, device, dataset='snli')
except KeyboardInterrupt:
    print("Graceful Exit")


======== Epoch 1 / 30 ========
Training...
  Batch    40  of    692.    Elapsed: 0:00:01.
  Batch    80  of    692.    Elapsed: 0:00:01.
  Batch   120  of    692.    Elapsed: 0:00:02.
  Batch   160  of    692.    Elapsed: 0:00:02.
  Batch   200  of    692.    Elapsed: 0:00:03.
  Batch   240  of    692.    Elapsed: 0:00:03.
  Batch   280  of    692.    Elapsed: 0:00:04.
  Batch   320  of    692.    Elapsed: 0:00:04.
  Batch   360  of    692.    Elapsed: 0:00:05.
  Batch   400  of    692.    Elapsed: 0:00:06.
  Batch   440  of    692.    Elapsed: 0:00:06.
  Batch   480  of    692.    Elapsed: 0:00:07.
  Batch   520  of    692.    Elapsed: 0:00:07.
  Batch   560  of    692.    Elapsed: 0:00:08.
  Batch   600  of    692.    Elapsed: 0:00:08.
  Batch   640  of    692.    Elapsed: 0:00:09.
  Batch   680  of    692.    Elapsed: 0:00:09.

  Average training loss: 0.69
  Training epcoh took: 0:00:09

Running Validation...
  Accuracy: 0.54
  Validation Loss: 0.68
  Validation took: 0:00:00

===